Before beginning this project, please visit [this](http://www.nutritionix.com/business/api) page and sign up for a free nutritionix account. The website says it may take 24 hours for your account to be confirmed, but mine was confirmed immediately.

# Nutrition

The USDA provides guidelines on proper nutrition for good health [this](http://health.gov/dietaryguidelines/dga2010/dietaryguidelines2010.pdf) report. See Appendix 5.

In this project, we will computes an optimal diet for meeting these guidelines. From Appendix 5, choose the category you find most appropriate or most interesting and look at the daily nutrient intake suggestions. For simplicity we will focus only on the following nutrients:

- Protein
- Carbohydrates
- Fiber
- Total Fat
- Saturated Fat
- Cholestorol 
- Calcium
- Iron
- Sodium
- Vitamin A
- Vitamin C
- Calories

*Hints:*

1. For some nutrients the suggestion is to consume *less than* a certain amount. These must be formulated as a constraint.
2. You will need to assume a daily calorie intake. There isn't a single answer to questions like this. Choose something you think is appropriate. You can use [this](http://healthyeating.sfgate.com/average-calorie-intake-human-per-day-versus-recommendation-1867.html) page as a guide, or search for your own answer.
3. For some nutrients, Nutritionix (our data source) will provide us with a "percent daily value" rather than an absolute amount in grams or miligrams. For these, assume that the provided percentage is applicable regardless of gender/age. Be sure to read the Nutritionix documentation.

### Problem Statement
Choose several foods which a person might consume in one day. Find the optimum number of servings to best meet the day's nutrional requirements.

### Useful links
The following documents were used to generate this solution:

1. Nutritionix [fields](https://docs.google.com/a/nutritionix.com/spreadsheet/ccc?key=0AmQ7yz5GxBrvdFhtRUpPdjl3VWk2U0dvZENyUVNrWGc&usp=drive_web#gid=0) - A google doc describing the fields available from the Nutritionix API
2. USDA [report](http://health.gov/dietaryguidelines/dga2010/dietaryguidelines2010.pdf) - A report outlining suggested daily intakes of nutrients. See Appendix 5.

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
import dill
import numpy as np
from scipy.optimize import fmin_slsqp

In [ ]:
# Load data downloaded from other sheet
with open('nutrient_dict.pkl','r') as f:
    nutrient_dict = dill.load(f)

In [ ]:
# Let's take a look at the data structure of nutrient_dict

# This loop shows all of the fields for a given food item 
for key,val in nutrient_dict["pizza"]['hits'][0]['fields'].items():
    print key, "  ...  ", val
print

# We can also access a single field
print nutrient_dict["pizza"]['hits'][0]['fields']['nf_saturated_fat']

In [ ]:
# In this cell I create a dict that maps each nutrient name to an index
# This makes my code easier to understand, because when I refer to a specific nutrient I 
# can index it by its nutrient name rather than an integer.
ls_nutrient_list = [
"Calories",
"Protein",
"Carbohydrates",
"Fiber",
"Calcium",
"Iron",
"Vitamin A",
]

ls_index_map = {}
for i,nutrient in enumerate(ls_nutrient_list):
    ls_index_map[nutrient] = i
print ls_index_map

In [ ]:
# Same type of dicts for constraints

# "Less than" constraints
constraint_lt_nutrient_list = [
    "Cals from fat",
    "Cals from saturated fat",
    "Cholesterol",
    "Sodium"
]
n_constraint_lt_nutrients = len(constraint_lt_nutrient_list)
constraint_lt_index_map = {}
for i,nutrient in enumerate(constraint_lt_nutrient_list):
    constraint_lt_index_map[nutrient] = i
print constraint_lt_index_map

# "Greater than" constraints
constraint_gt_nutrient_list = [
    "Cals from fat",
    "Vitamin C"
]
n_constraint_gt_nutrients = len(constraint_gt_nutrient_list)
constraint_gt_index_map = {}
for i,nutrient in enumerate(constraint_gt_nutrient_list):
    constraint_gt_index_map[nutrient] = i
print constraint_gt_index_map

In [ ]:
# Here I create a map from the names I use to the names used by the Nutritionix API
nf_map = {
"Calories":"nf_calories",
"Protein":"nf_protein",
"Carbohydrates":"nf_total_carbohydrate",
"Fiber":"nf_dietary_fiber",
"Cals from fat":"nf_calories_from_fat",
"Cals from saturated fat":"nf_saturated_fat",      # This is a special case! nf_saturated_fat is in grams
"Cholesterol":"nf_cholesterol",
"Calcium":"nf_calcium_dv",
"Iron":"nf_iron_dv",
"Sodium":"nf_sodium",
"Vitamin A":"nf_vitamin_a_dv",
"Vitamin C":"nf_vitamin_c_dv",
    }

In [ ]:
# target vector
calories     = 2500.

requirements = np.zeros([len(ls_nutrient_list)])
requirements[ls_index_map['Calories']] = calories                 # calories
requirements[ls_index_map['Protein']] = 56.                       # grams
requirements[ls_index_map['Carbohydrates']] = 130.                # grams
requirements[ls_index_map['Fiber']] = 34.                         # grams
requirements[ls_index_map['Calcium']] = 100.                      # Percent DV
requirements[ls_index_map['Iron']] = 100.                         # Percent DV
requirements[ls_index_map['Vitamin A']] = 100.                    # Percent DV

In [ ]:
# Constraints vectors
constraints_lt = np.zeros([len(constraint_lt_nutrient_list)])
constraints_lt[constraint_lt_index_map['Cals from fat']] = calories*.35
constraints_lt[constraint_lt_index_map['Cals from saturated fat']] = calories*.1
constraints_lt[constraint_lt_index_map['Cholesterol']] = 300
constraints_lt[constraint_lt_index_map['Sodium']] = 2300

constraints_gt = np.zeros([len(constraint_gt_nutrient_list)])
constraints_gt[constraint_gt_index_map['Cals from fat']] = calories*.20
constraints_gt[constraint_gt_index_map['Vitamin C']] = 100

In [ ]:
# Let's grab some foods from our dictionary
#food_list = ["spinach", "quinoa","tomato","balsamic vinaigrette","brown rice","blueberries","pizza","banana"]
food_list = ["spinach","quinoa","tomato","balsamic vinaigrette","brown rice","blueberries",
         "fried rice", "pizza", "naan", "fried chicken", "maggi noodles", "chicken tikka masala",
         "nasi lemak", "white bread", "salmon", "french fries", "banana"
        ] 

In [ ]:
# This function picks a nutrient out of the nutrient dictionary
# Nutritionix returns NaN for some values. These will be returned as zero.
def pick_nutrient(nutrient_dict, food, field):
    out = nutrient_dict[food]['hits'][0]['fields'][field]
    
    if field=='nf_saturated_fat':

        saturated_fat_grams = out
        total_fat_grams = pick_nutrient(nutrient_dict, food, 'nf_total_fat')
        calories = pick_nutrient(nutrient_dict, food, 'nf_calories')
        out = saturated_fat_grams/total_fat_grams * calories

    if out:
        return out
    else:
        return 0.

In [ ]:
# This function builds the "food matrix"
def build_food_matrix(foods, nutrient_list, nutrient_dict, index_map, nf_map):
    matrix = np.matrix(np.zeros([len(nutrient_list),len(foods)]))
    for jfood,food in enumerate(foods):
        for nutrient in nutrient_list:
            matrix[index_map[nutrient],jfood] = pick_nutrient(nutrient_dict,food,nf_map[nutrient])
    return matrix

In [ ]:
# Matrix for least-squares minimization
food_matrix_ls = build_food_matrix(food_list, 
                                   ls_nutrient_list, 
                                   nutrient_dict, 
                                   ls_index_map, 
                                   nf_map)

# Matrix for "Less than" constraints
food_matrix_constraint_lt = build_food_matrix(food_list, 
                                   constraint_lt_nutrient_list, 
                                   nutrient_dict, 
                                   constraint_lt_index_map, 
                                   nf_map)

# Matrix for "Greater than" constraints
food_matrix_constraint_gt = build_food_matrix(food_list, 
                                   constraint_gt_nutrient_list, 
                                   nutrient_dict, 
                                   constraint_gt_index_map, 
                                   nf_map)

# Set to True to print the nutrients in food list
if (False):
    for i,food in enumerate(food_list):
        print food
        for nutrient in ls_nutrient_list:
            print nutrient, food_matrix_ls[ls_index_map[nutrient],i]
        print

In [ ]:
# This block of code is merely a sanity check
# This is a _very_ useful thing to do in general! We want to make sure that our numbers are reasonable
# It's very easy to make a mistake. 
# For example if Nutritionix specified something in mg but USDA specified in grams, we might catch it here.

ones_vector = np.matrix(np.ones([len(food_list),1]))
one_of_each = food_matrix_ls*ones_vector              # Assume we're having 1 serving of each food item

print "Least-squares nutrients"
for i, nutrient in enumerate(ls_nutrient_list):
    print nutrient, float(one_of_each[i][0]), "/", requirements[i]
    
print
print '"Less than" constraints:'
one_of_each_lt = food_matrix_constraint_lt*ones_vector
for i, nutrient in enumerate(constraint_lt_nutrient_list):
    print nutrient, float(one_of_each_lt[i][0]), "/", constraints_lt[i],"---", constraints_lt[i]-float(one_of_each_lt[i][0])
    
print
print '"Greater than" constraints:'
one_of_each_gt = food_matrix_constraint_gt*ones_vector
for i, nutrient in enumerate(constraint_gt_nutrient_list):
    print nutrient, float(one_of_each_gt[i][0]), "/", constraints_gt[i],"---", float(one_of_each_gt[i][0])-constraints_gt[i]

In [ ]:
# Objective function for optimization

def obj0(x, food_matrix, requirements):
    x = np.matrix(x).reshape(len(x),1)
    return np.linalg.norm(food_matrix*x - requirements)
obj = lambda x:obj0(x,food_matrix_ls,requirements)

In [ ]:
# Constraint function

def constraints0(x, food_matrix_constraint_lt, constraints_lt, food_matrix_constraint_gt, constraints_gt):
    # In final results, the return value of this function must be all >=0
    
    x = np.matrix(x).reshape(len(x),1)
    constraints_lt = np.matrix(constraints_lt).reshape(len(constraints_lt),1)
    constraints_gt = np.matrix(constraints_gt).reshape(len(constraints_gt),1)
    
    retval_lt = (constraints_lt-food_matrix_constraint_lt*x)
    retval_gt = (food_matrix_constraint_gt*x-constraints_gt)

    retval = retval_lt.tolist()
    retval.extend(retval_gt.tolist())
    
    return np.array(retval).reshape(len(retval))

constraints = lambda x : constraints0(x,food_matrix_constraint_lt, constraints_lt, food_matrix_constraint_gt, constraints_gt)

In [ ]:
# Another sanity check (for datatypes)
print obj(ones_vector)
print constraints(ones_vector)

In [ ]:
# Bounds for the design variable (number of servings of each food item)
bounds = [(0,10)]*len(food_list)
print bounds

In [ ]:
# Run!
optimization_output = fmin_slsqp(obj,                   # Objective function
                                 ones_vector,           # Initial guess
                                 f_ieqcons=constraints, # Constraints function
                                 bounds=bounds,
                                 iprint = 0
                                )

In [ ]:
print optimization_output

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

In [ ]:
locs = np.array(range(len(food_list)))

plt.bar(locs-.2,optimization_output,width=0.4)
plt.xticks(locs, food_list, rotation=90)
plt.ylabel("Number of servings")
""

In [ ]:
locs = np.array(range(len(ls_nutrient_list)))

x = np.matrix(optimization_output).reshape([len(food_list),1])
b = food_matrix_ls*x

b_percentage = np.array(b).reshape((7,)) / requirements * 100.

plt.bar(locs-.2,b_percentage,width=0.4)
plt.plot([-1,len(locs)],[100.,100],'k-')
plt.xticks(locs, ls_nutrient_list, rotation=45)
plt.ylabel('Nutrition achieved at optimum servings (of % DV)')
""

In [ ]:
locs = np.array(range(len(constraint_lt_nutrient_list)))

x = np.matrix(optimization_output).reshape([len(food_list),1])
b = food_matrix_constraint_lt*x

plt.bar(locs-.2,b,width=0.4)
plt.plot(locs,constraints_lt,'rv')
plt.xticks(locs, constraint_lt_nutrient_list, rotation=45)
plt.ylabel('"Less than" constraints')
""

In [ ]:
locs = np.array(range(len(constraint_gt_nutrient_list)))

x = np.matrix(optimization_output).reshape([len(food_list),1])
b = food_matrix_constraint_gt*x

plt.bar(locs-.2,b,width=0.4)
plt.plot(locs,constraints_gt,'r^')
plt.xticks(locs, constraint_gt_nutrient_list, rotation=45)
plt.ylabel('"Greater than" constraints')
""

### Attribution
Powered by [Nutritionix API](http://www.nutritionix.com/api)